[Reference](https://blog.det.life/dont-write-another-line-of-code-until-you-see-these-pydantic-v2-breakthrough-features-5cdc65e6b448)

# 1. Validating Functions

In [1]:
from pydantic import validate_call
from pydantic.types import conint


@validate_call
def echo_hello(n_times: conint(gt=0, lt=11), name: str, loud: bool):
    """
    Greets someone with an echo.

    Args:
        n_times: How many echos. Min value is 1, max is 10.
        name: Name to greet
        loud: Do you want the greeting to be loud?
    """
    greeting = f"Hello {name}!"

    if loud:
        greeting = greeting.upper() + "!!"

    for i in range(n_times):
        print(greeting)


# Call this function
echo_hello(n_times=1, name="Yaakov", loud=True)   # Valid
echo_hello(n_times=10, name="Yaakov", loud=True)  # Valid

# The following will raise an error:
echo_hello(n_times=20, name="Yaakov", loud=True)  # Invalid!
echo_hello(n_times=1, name=1234, loud=True)       # Invalid!

HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!
HELLO YAAKOV!!!


ValidationError: ignored

# 2. Discriminated Unions


In [2]:
#from typing import Union, Literal, List

from pydantic import BaseModel, Field


class ModelA(BaseModel):
    d_type: Literal["single"]
    value: int = Field(default=0)


class ModelB(ModelA):
    """Inherits from ModelA, making the union challenging"""
    d_type: Literal["many"]
    values: List[int] = Field(default_factory=list)


class ModelC(BaseModel):
    v: Union[ModelA, ModelB] = Field(discriminator="d_type")


# Populate with extra fields, see what happens
m_1 = ModelC(v={"value": 123, "values": [123], "d_type": "single"})
m_2 = ModelC(v={"value": 123, "values": [123], "d_type": "many"})

print(m_1, m_2, sep="\n")
# v=ModelA(d_type='single', value=123)
# v=ModelB(d_type='many', value=123, values=[123])

NameError: ignored

# 3. Validated Types with Annotated Validators

In [3]:
from typing_extensions import Annotated
from pydantic.functional_validators import AfterValidator


def validate(v: int):
    assert v > 0

PositiveNumber = Annotated[int, AfterValidator(validate)]

In [4]:
from typing import Any
from typing_extensions import Annotated

from pydantic import BaseModel
from pydantic.functional_validators import AfterValidator, BeforeValidator


def remove_currency(v: Any) -> int:
    """Remove currency symbol from any input"""
    if isinstance(v, str):
        v = v.replace('$', '')
    return v

def truncate_max_number(v: int) -> int:
    """Any number greater than 100 will be set at 100"""
    return min(v, 100)


# Create a custom type (importable!)
Price = Annotated[
    int,
    BeforeValidator(remove_currency),
    AfterValidator(truncate_max_number)
]


class Model(BaseModel):
    price: Price


# Instantiate the model to demonstrate
m = Model(price="$12")      # price=12
m = Model(price=12)         # price=12
m = Model(price=101)        # price=100
print(m)

price=100


# 4. Validation without BaseModel using TypeAdapter

In [5]:
from typing import List, Any
from typing_extensions import Annotated

import pytest

from pydantic import TypeAdapter
from pydantic.functional_validators import BeforeValidator


def coerce_to_list(v: Any) -> List[Any]:
    if isinstance(v, list):
        return v
    else:
        return [v]


NumberList = Annotated[
    List[int],
    BeforeValidator(coerce_to_list)
]


@pytest.mark.parameterize(
    ('v', 'expected'),
    [
        pytest.param(1, [1], id="single to list"),
        pytest.param([1, 2, 3], [1, 2, 3], id="list, no change"),
        pytest.param([1, '2'], [1, 2], id="list with string nums"),
    ]
)
def test_number_list(v: Any, expected: List[int]):
    ta = TypeAdapter(NumberList)
    res = ta.validate_python(v)
    assert res == expected

# 5. Custom Serialization

In [6]:
from datetime import datetime
from pydantic import BaseModel, field_serializer

class BroadwayTicket(BaseModel):
    show_name: str
    show_time: datetime

    @field_serializer("show_time")
    def transform_show_time(v) -> str:
        """Returns human readable show time format"""
        return v.strftime("%b %d, %Y, %I:%M %p")


# Create an object
my_tickets = BroadwayTicket(
    show_name="Parade",
    show_time=datetime(2023, 8, 5, 19)  # August 8, 7:00PM
)

print(my_tickets.model_dump())

{'show_name': 'Parade', 'show_time': 'Aug 05, 2023, 07:00 PM'}
